In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import wavelet_transform as wt
import signal_processing as sp
from create_graph import create_graph
import scipy.sparse


/tmp/ipykernel_530350/4102537558.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
%matplotlib inline

In [3]:
def precompute_wavelet(poly_division, time_interval):
    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    features = df.columns[2:]
    try:
        adj_matrix = np.load(f"data/adj_matrix/{poly_division}.npy")
    except:
        adj_matrix = scipy.sparse.load_npz(f"data/adj_matrix/{poly_division}.npz").toarray()
    signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
    df_res = df[["id_poly", "date"]].copy()

    for k, feature in enumerate(features):
        signal = signal_multi[:, :, k]

        n_timestamps = signal_multi.shape[1]

        wav =  wt.WaveletTransform(
            adj_matrix,
            n_timestamps,
            graph_product="strong",
            n_filters=32,
            kernel="abspline",
            scaling_function=False,
            method="chebyshev",
            order_chebyshev=30,
        )
        coeffs = wav.transform(signal)
        coeffs = sp.get_scaled_coefficients(coeffs)
        # create column for coeffs in the df_res
        for i in range(coeffs.shape[-1]):
            df_res[f"{feature}_coeff_{i}"] = coeffs[:, :, i].flatten()
        df_res = df_res.copy()
        df_res[df_res.columns[2:]] = df_res[df_res.columns[2:]].astype(np.float32)
        df_res.to_csv(f"data/coeffs/{feature}_{poly_division}_{time_interval}.csv", index=False)
        df_res = df[["id_poly", "date"]].copy()

In [4]:
for time_interval in ["Month"]:
    for poly_division in ["SpGrid", "SpDistricts", "SpCenterCensus10k", "SpCenterCensus5k"]:
        precompute_wavelet(poly_division, time_interval)

/tmp/ipykernel_530350/278219420.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
/home/giovani/Documents/AutoSatViewer/wavelet_code/signal_processing.py:145: RuntimeWarning: invalid value encountered in divide
  scaled_coefficients[:, i] = np.log(scaled_coefficients[:, i] + 1) / np.log(
/tmp/ipykernel_530350/278219420.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclud